In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# ! pip install flask
# ! pip install pyngrok

In [2]:
! killall ngrok
! ngrok config add-authtoken 2QN9JlBY91WFv2EKuaFUCPCBsJC_7cK8YoEkQbjHj3pnnNpD5

'killall' is not recognized as an internal or external command,
operable program or batch file.


Authtoken saved to configuration file: C:\Users\Charan A A\AppData\Local/ngrok/ngrok.yml


In [3]:
import os
import threading
from PIL import Image
import google.generativeai as genai
from dotenv import load_dotenv
import os
import io
from flask import Flask, render_template
from pyngrok import ngrok, conf
from flask import request

# Load environment variables from the .env file
load_dotenv()
# Access the environment variable
api_key = os.getenv("OPENAI_API_KEY")



app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

model = genai.GenerativeModel('gemini-pro-vision')


# Define Flask routes
@app.route("/", methods=['GET', 'POST'])
def index():
    
    global chat,model,api_key
    genai.configure(api_key=api_key)

    
    # content_type = request.headers.get('Content-Type')

    if request.args.get('question'):

        query = request.args.get("question")
        img_bytes = request.args.get("image")
        img = Image.open(io.BytesIO(img_bytes))

        
        safety_ratings = {
        'HARM_CATEGORY_SEXUALLY_EXPLICIT':'block_none',
        'HARM_CATEGORY_HATE_SPEECH': 'block_none',
        'HARM_CATEGORY_HARASSMENT': 'block_none',
        'HARM_CATEGORY_DANGEROUS_CONTENT' : 'block_none'
        }
                           
        response = model.generate_content([query, img])
        response = response.text
        
        print(response)
        
        return response
        
    else:
        return 'Error'

if __name__ == '__main__':
    app.run()

d:\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 * ngrok tunnel "https://fc70-103-196-28-194.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Feb/2024 12:25:36] "POST / HTTP/1.1" 200 -


api for image and text

In [1]:
import textract

def read_text_from_file(file_path):
    try:
        text = textract.process(file_path).decode('utf-8')
        return text
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return ""

# Example usage:
file_paths = [r'D:\Downloads\events.pdf']

for file_path in file_paths:
    text_content = read_text_from_file(file_path)
    print(f"Content of {file_path}:\n{text_content}\n{'-'*30}")


Error reading file D:\Downloads\events.pdf: The command `pdftotext D:\Downloads\events.pdf -` failed with exit code 127
------------- stdout -------------
------------- stderr -------------

Content of D:\Downloads\events.pdf:

------------------------------


In [4]:
import PyPDF2
from pptx import Presentation
from docx import Document

def read_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        text = ''
        for page_number in range(pdf_reader.numPages):
            page = pdf_reader.getPage(page_number)
            text += page.extractText()
    return text

def read_ppt(ppt_path):
    prs = Presentation(ppt_path)
    text = ''
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text += shape.text + '\n'
    return text

def read_docx(docx_path):
    doc = Document(docx_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def read_txt(txt_path):
    with open(txt_path, 'r', encoding='utf-8') as txt_file:
        text = txt_file.read()
    return text

def read_documents(doc_paths):
    text = ''
    for doc_path in doc_paths:
        if doc_path.endswith('.pdf'):
            text += read_pdf(doc_path)
        elif doc_path.endswith('.pptx'):
            text += read_ppt(doc_path)
        elif doc_path.endswith('.docx'):
            text += read_docx(doc_path)
        elif doc_path.endswith('.txt'):
            text += read_txt(doc_path)
        else:
            print(f"Unsupported file format for: {doc_path}")
    return text

# Example usage:
document_paths = ['D:\Downloads\events.pdf']
text_content = read_documents(document_paths)
print(text_content)


DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.